# Roles

### Put Your OPENROUTER_API_KEY here

In [ ]:
import os
os.environ["OPENROUTER_API_KEY"] = "paste_your_api_key_here"

In [ ]:
!pip3 install langchain langchain_openai

In [ ]:
import os
from langchain_openai import ChatOpenAI

try:
    model_name
except NameError:
    model_name="openai/gpt-4o-mini"

print("Model Name:", model_name)
print("Provider:", "OpenRouter AI")

llm=ChatOpenAI(model_name=model_name,
               openai_api_key=os.environ.get("OPENROUTER_API_KEY"),
               openai_api_base="https://openrouter.ai/api/v1")

In [ ]:
import textwrap
from langchain_core.prompts import ChatPromptTemplate

from IPython.display import display, clear_output, Markdown
from ipywidgets import widgets, Layout

conversation_output = widgets.Output()
messages = []

def run_chatbot(system_prompt, initial_message):
    global messages 
    messages = [ {'role':'system', 'content': system_prompt} ]
    conversation_output.clear_output()

    messages.append({'role': 'assistant', 'content': initial_message})

    text_input = widgets.Text(
        placeholder='Type your message here...',
        layout=widgets.Layout(width='50%')
    )
    submit_button = widgets.Button(description="Send")

    input_box = widgets.HBox([text_input, submit_button])
    display(conversation_output, input_box)

    def on_submit_click(b):
        message = text_input.value
        text_input.value = ''  # Clear the input field

        with conversation_output:
            display(Markdown(f"**User**: {message}"))
            messages.append({'role': 'user', 'content': message})
            response = get_completion_messages(messages)
            display(Markdown(f"**AI**: {response}"))
            messages.append({'role': 'assistant', 'content': response})

    submit_button.on_click(on_submit_click)

    # Display initial AI message
    with conversation_output:
        display(Markdown(f"**AI**: {initial_message}"))

def wrap_text(text, max_width=80):
    """
    Wraps the text to the specified max_width, preserving line breaks and formatting.
    """
    text = text.lstrip()
    lines = text.splitlines()  # Split the text into lines
    wrapped_lines = []
    for line in lines:
        if line.strip():  # Skip empty lines
            wrapped_line = textwrap.fill(line, max_width, initial_indent='', subsequent_indent='')
            wrapped_lines.extend(wrapped_line.splitlines())  # Preserve line breaks
        else:
            wrapped_lines.append('')  # Keep empty lines
    return '\n'.join(wrapped_lines)

def print_messages():
    for message in messages:
        role = message['role']
        content = message['content']
        
        if role == 'system':
            print("System:")
            print("-" * 40)
            print(content)
        elif role == 'user':
            print("User: ", end="")
            print(wrap_text(content))
        elif role == 'assistant':
            print("Assistant: ", end="")
            print(wrap_text(content))
        print()  # Add an extra newline for spacing

def print_prompt_and_response(prompt, response):
    print("Prompt: ")
    print(wrap_text(prompt))
    print("")
    print("Response: ")
    print(response)

def print_messages_and_response(messages, response):
    prompt = ChatPromptTemplate(messages=messages)
    print_prompt_and_response(prompt.format(), response)

def get_completion(prompt, temperature=0.0):
    response = llm.invoke(prompt, temperature=temperature)
    wrapped_response = wrap_text(response.content)
    return wrapped_response

def get_completion_messages(messages, temperature=0.0):
    response=llm.invoke(messages, temperature=temperature)
    wrapped_response = wrap_text(response.content)
    return wrapped_response

Many times it helps to give the LLM a role or a persona. That helps set
the context for how it is meant to respond. This is a very common thing
to do and is why you see many prompts that start with:

-   You are an expert at …
-   You are a professional …

This instructions ground the LLMs responses with information about how
it should act and respond.

## Act Like a Mouse

We can tell the LLM that it is to act like a mouse.

In [ ]:
prompt="""
You are to act like a mouse.
What do you think of skateboards?
"""
response=get_completion(prompt)
print(response)

## Act Like a Kid

In [ ]:
prompt="""
You are a kid.
What do you think of skateboards?
"""
response=get_completion(prompt)
print(response)

## Is a Married Person Looking at an Unmarried Person?

In this part we ask the LLM to solve a logical problem. We will ask it
to solve this problem twice. First without prompting it with a role.
Then the second time we will ask it to answer by giving it a role and
asking it to think about all the cases.

So here is our question:

> Jack is looking at Anne. Anne is looking at George. Jack is married,
> George is not, and we don’t know if Anne is married. Is a married
> person looking at an unmarried person?

The answer to our question is yes. And the way to see it is to just
think about the cases:

-   If Anne is married, then Anne (married) is looking at George
    (unmarried)
-   If Anne is unmarried, then Jack (married) is looking at Anne
    (unmarried)

So in both cases the answer is yes, a married person is looking at an
unmarried person.

Now first we just try to have it solve the problem without any
additional prompting about a role.

In [ ]:
prompt="""
Jack is looking at Anne. Anne is looking at George. Jack is married, George is
not, and we don’t know if Anne is married. Is a married person looking at an
unmarried person?
"""
response=get_completion(prompt)
print(response)

The answer above is right, but the LLM thinks its true since Jack is
looking at George. Which is not quite correct.

## Act Like a Professor of Logic

But now we tell the LLM its a professor of logic and it should think
through all the possibilities. With many LLMs this gets them closer to
the right solution and sometimes they get it completly right with this
kind of prompting.

Asking a model to think “step-by-step” or by listing its reasoning is
called “Chain of Thought(CoT)” prompting.

In [ ]:
prompt="""
You are a smart professor of logic who can answer complex logic problems. Think
through all the cases before you answer.

Logic Problem:
Jack is looking at Anne. Anne is looking at George. Jack is
married, George is not, and we don’t know if Anne is married. Is a married
person looking at an unmarried person?
"""

response=get_completion(prompt)
print(response)

This time the LLM does much better and you see it doing better thinking
step-by-step.

## Is This Equation Solved Correctly?

Here is another example involving a simple algebra problem.

Without further prompting we ask the LLM if this (incorrect) problem
solution is correct.

In [ ]:
prompt ="""
Is this equation solved correctly below?

2x-3=9
2x=6
x=3
"""
response=get_completion(prompt)
print(response)

Hmm…it seems to think the solution is correct. It didn’t see anything
wrong with the solution.

## You are a Teacher of Math. Reason Step-By-Step

Now we ask it again, but this time we tell it that its a teacher of
math.

In [ ]:
prompt = """
You are a teacher of math. And can reason step-by-step to see if something is
correct. First work out your own solution to the problem. Then carefully
consider how to answer the following question:

Is this problem solved correctly below?
2x-3=9
2x=6
x=3
"""
response=get_completion(prompt)
print(response)

Here it does much better in the second attempt. It still says the
problem is solved correctly originally, but then it finds that there is
a mistake. Definitely the step-by-step reasoning it is doing is an
improvement over its first answer.